## 参考资料

* ndarray文档  
[https://mxnet.incubator.apache.org/api/python/ndarray.html](https://mxnet.incubator.apache.org/api/python/ndarray.html)

* yield用法  
[https://www.ibm.com/developerworks/cn/opensource/os-cn-python-yield/](https://www.ibm.com/developerworks/cn/opensource/os-cn-python-yield/)

In [7]:
# 导入mxnet的ndarray, autograd
from mxnet import autograd
from mxnet import ndarray as nd

## 创建数据集

In [23]:
# 训练数据的维度
num_inputs = 2

# 训练数据的样本数量
num_examples = 1000

# 实际的权重w
true_w = [2, -3.4]

# 实际的偏置b
true_b = 4.2

# 随机生成均值为0, 方差为1, 服从正态分布的训练数据X, 
X = nd.random_normal(shape=(num_examples, num_inputs))

# 根据X, w, b生成对应的输出y
y = true_w[0] * X[:, 0] + true_w[1] * X[:, 1] + true_b 

# 给y加上随机噪声
y = 0.01 * nd.random_normal(shape=y.shape)

## 数据读取

In [16]:
import random

# 训练时的批数据大小
batch_size = 10

# 通过yield进行数据读取
def data_iter():
    # 产生样本的索引
    idx = list(range(num_examples))
    # 将索引随机打乱
    random.shuffle(idx)
    # 迭代一个epoch, xrange循环时效率比range更高
    for i in xrange(0, num_examples, batch_size):
        # 依次取出样本的索引, 这种实现方式在num_examples/batch_size不能整除时也适用
        j = nd.array(idx[i:min((i + batch_size), num_examples)])
        # 根据提供的索引取元素
        yield nd.take(X, j), nd.take(y, j)

In [27]:
# 查看data_iter是否是generator函数
from inspect import isgeneratorfunction 
print isgeneratorfunction(data_iter)

# data_iter类似于类的定义, 而data_iter()相当于一个类的实例, 当然是匿名实例
import types 
print isinstance(data_iter(), types.GeneratorType)

# 读取数据测试
for data, label in data_iter():
    print(data, label)
    break

True
True
(
[[-1.46144283 -2.03927684]
 [ 0.03402483 -0.03641445]
 [-0.47866404  0.10702948]
 [ 0.39163432 -0.997118  ]
 [ 0.32307819 -0.66136807]
 [ 2.21568346 -0.32404178]
 [ 0.09458331 -1.92368269]
 [-0.61783087 -0.58341485]
 [-0.53721207 -0.33021879]
 [-0.31719905  0.25661287]]
<NDArray 10x2 @cpu(0)>, 
[-0.00857094 -0.01071989 -0.00500661  0.0048739   0.00531892 -0.00569082
 -0.00541457 -0.01004949 -0.01156097  0.01234144]
<NDArray 10 @cpu(0)>)


## 初始化模型参数

In [28]:
# 随机初始化权重w
w = nd.random_normal(shape=(num_inputs, 1))
# 偏置b初始化为0
b = nd.zeros((1,))
# w, b放入list里
params = [w, b]

# 需要计算反向传播, 添加自动求导
for param in params:
    param.attach_grad()

## 定义模型

In [ ]:
# 定义运算y = w * x + b
def net(X):
    # 向量, 矩阵乘用dot
    return nd.dot(X, w) + b

## 损失函数

In [29]:
# 定义平方损失
def square_loss(yhat, y):
    # 注意这里我们把y变形成yhat的形状来避免矩阵形状的自动转换
    return (yhat - y.reshape(yhat.shape)) ** 2

## 优化

In [ ]:
# 定义随机梯度下降法
def SGD(params, lr):
    # 对参数进行梯度下降
    for param in params:
        # 这样写不会创建新的param, 而是会写好原来的param里
        param[:] = param - lr * param.grad

In [ ]:
# 使用sklearn的波士顿数据集
from sklearn.datasets import load_boston